1. Extract deep features from the pre-trained VGG-16 model and extract hand crafted
   Histogram Oriented Gradient (HOG) features for MNIST dataset. Stack the deep
   features with HOG features and model it using a random forest classifier to classify
   the MNIST dataset. Run the hybrid model 5 times and compute the mean accuracy.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as gb
import cv2
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , Dropout , Flatten , MaxPooling2D , BatchNormalization ,experimental
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras.models import Model

In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import mnist
from skimage.feature import hog
import tensorflow as tf

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0  # Normalize pixel values to [0, 1]

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [3]:
# Load pre-trained VGG-16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))  # MNIST images are 28x28, so resize them to 32x32
vgg_model.trainable = False  # Freeze the weights

def extract_deep_features(images):
    print('Extracting deep features')
    images = np.expand_dims(images, axis=-1)  # Add channel dimension for grayscale images
    images = np.repeat(images, 3, axis=-1)  # Convert grayscale to RGB
    images = np.array([tf.image.resize(image, (32, 32)) for image in images])  # Resize images to 32x32
    deep_features = vgg_model.predict(images)
    return deep_features

def extract_hog_features(images):
    print('Extracting deep features')
    hog_features = []
    for image in images:
        hog_feature = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
        hog_features.append(hog_feature)
    return np.array(hog_features)

# Extract deep features for training and test sets
X_train_deep_features = extract_deep_features(X_train)
X_test_deep_features = extract_deep_features(X_test)

# Extract HOG features for training and test sets
X_train_hog_features = extract_hog_features(X_train)
X_test_hog_features = extract_hog_features(X_test)

# Stack deep features with HOG features
X_train_stacked = np.concatenate((X_train_deep_features.reshape(len(X_train_deep_features), -1), X_train_hog_features), axis=1)
X_test_stacked = np.concatenate((X_test_deep_features.reshape(len(X_test_deep_features), -1), X_test_hog_features), axis=1)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Run the hybrid model 5 times and compute the mean accuracy
accuracies = []
for _ in range(2):
    # Split the data into training and validation sets
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_stacked, y_train, test_size=0.2, random_state=42)
    
    # Train the model
    print('Fitting rf model')
    rf_classifier.fit(X_train_split, y_train_split)
    
    # Evaluate the model on the validation set
    val_predictions = rf_classifier.predict(X_val)
    print('Getting predictions')
    accuracy = accuracy_score(y_val, val_predictions)
    accuracies.append(accuracy)

mean_accuracy = np.mean(accuracies)
print("Mean accuracy:", mean_accuracy)

Extracting deep features
1875/1875 [==============================] - 162s 86ms/step
Extracting deep features
313/313 [==============================] - 44s 142ms/step
Extracting deep features
Extracting deep features
Fitting rf model
Getting predictions
Fitting rf model
Getting predictions
Mean accuracy: 0.895875


In [4]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Load the pre-trained VGG-16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
model.trainable = False

# Function to extract deep features from VGG-16
def extract_deep_features(images):
    print('Extracting deep features')
    images = np.expand_dims(images, axis=-1)  # Add channel dimension for grayscale images
    images = np.repeat(images, 3, axis=-1)  # Convert grayscale to RGB
    images = np.array([tf.image.resize(image, (32, 32)) for image in images])  # Resize images to 32x32
    deep_features = vgg_model.predict(images)
    return deep_features

# Function to extract SIFT features

def extract_sift_features(images):
    print('Extracting SIFT features')
    sift = cv2.SIFT_create()
    max_features = 0
    sift_features = []
    for image in images:
        kp, des = sift.detectAndCompute(image, None)
        if des is None:
            des = np.zeros((0, 128))  # Use zeros if no keypoints found
        sift_features.append(des)
        max_features = max(max_features, len(des))
    
    # Pad features to make them all the same length
    for i in range(len(sift_features)):
        if len(sift_features[i]) < max_features:
            pad_width = ((0, max_features - len(sift_features[i])), (0, 0))
            sift_features[i] = np.pad(sift_features[i], pad_width, mode='constant')
    
    return np.array(sift_features)


# Extract deep features for training and test sets
x_train_deep_features = extract_deep_features(x_train)
x_test_deep_features = extract_deep_features(x_test)

# Extract SIFT features for training and test sets
x_train_sift_features = extract_sift_features(x_train)
x_test_sift_features = extract_sift_features(x_test)

# Stack deep features with SIFT features
x_train_stacked = np.concatenate((x_train_deep_features.reshape(len(x_train_deep_features), -1),
                                  x_train_sift_features.reshape(len(x_train_sift_features), -1)), axis=1)
x_test_stacked = np.concatenate((x_test_deep_features.reshape(len(x_test_deep_features), -1),
                                 x_test_sift_features.reshape(len(x_test_sift_features), -1)), axis=1)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Run the hybrid model 5 times and compute the mean accuracy
accuracies = []
loop =1
for i in range(loop):
    # Split the data into training and validation sets
    x_train_split, x_val, y_train_split, y_val = train_test_split(x_train_stacked, y_train, test_size=0.2, random_state=42)

    # Train the model
    print('Fitting rf model')
    rf_classifier.fit(x_train_split, y_train_split)

    # Evaluate the model on the validation set
    print('Getting predictions')
    val_predictions = rf_classifier.predict(x_val)
    accuracy = accuracy_score(y_val, val_predictions)
    print('Accuracy:', accuracy)
    accuracies.append(accuracy)

mean_accuracy = np.mean(accuracies)
print("Mean accuracy:", mean_accuracy)

Extracting deep features
1875/1875 [==============================] - 227s 121ms/step
Extracting deep features
313/313 [==============================] - 28s 89ms/step
Extracting SIFT features
Extracting SIFT features
Fitting rf model
Getting predictions
Accuracy: 0.9135
Mean accuracy: 0.9135


In [5]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Load the pre-trained VGG-16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
model.trainable = False

# # Function to extract deep features from VGG-16
# def extract_deep_features(images):
#     print('Extracting deep features')
#     images = np.expand_dims(images, axis=-1)  # Add channel dimension for grayscale images
#     images = np.repeat(images, 3, axis=-1)  # Convert grayscale to RGB
#     images = np.array([tf.image.resize(image, (32, 32)) for image in images])  # Resize images to 32x32
#     deep_features = vgg_model.predict(images)
#     return deep_features

# # Function to extract SIFT features
# def extract_sift_features(images):
#     sift = cv2.SIFT_create()
#     sift_features = []
#     for image in images:
#         kp, des = sift.detectAndCompute(image, None)
#         sift_features.append(des.flatten() if des is not None else np.zeros(128))  # Use zeros if no keypoints found
#     return np.array(sift_features)

# # Function to extract HOG features
# def extract_hog_features(images):
#     hog_features = []
#     for image in images:
#         hog_feature = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
#         hog_features.append(hog_feature)
#     return np.array(hog_features)

# Extract deep features for training and test sets
x_train_deep_features = extract_deep_features(x_train)
x_test_deep_features = extract_deep_features(x_test)

# Extract SIFT features for training and test sets
x_train_sift_features = extract_sift_features(x_train)
x_test_sift_features = extract_sift_features(x_test)

# Extract HOG features for training and test sets
x_train_hog_features = extract_hog_features(x_train)
x_test_hog_features = extract_hog_features(x_test)


# Stack deep features with SIFT and HOG features
x_train_stacked = np.concatenate((x_train_deep_features.reshape(len(x_train_deep_features), -1),
                                  x_train_sift_features.reshape(len(x_train_sift_features), -1),
                                  x_train_hog_features.reshape(len(x_train_hog_features), -1)), axis=1)
x_test_stacked = np.concatenate((x_test_deep_features.reshape(len(x_test_deep_features), -1),
                                 x_test_sift_features.reshape(len(x_test_sift_features), -1),
                                 x_test_hog_features.reshape(len(x_test_hog_features), -1)), axis=1)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Run the hybrid model 5 times and compute the mean accuracy
accuracies = []
for _ in range(1):
    # Split the data into training and validation sets
    x_train_split, x_val, y_train_split, y_val = train_test_split(x_train_stacked, y_train, test_size=0.2, random_state=42)

    # Train the model
    print('Fitting rf model')
    rf_classifier.fit(x_train_split, y_train_split)

    # Evaluate the model on the validation set
    print('Getting predictions')
    val_predictions = rf_classifier.predict(x_val)
    accuracy = accuracy_score(y_val, val_predictions)
    print('Accuracy:', accuracy)
    accuracies.append(accuracy)

mean_accuracy = np.mean(accuracies)
print("Mean accuracy:", mean_accuracy) 

Extracting deep features
1875/1875 [==============================] - 267s 142ms/step
Extracting deep features
313/313 [==============================] - 42s 133ms/step
Extracting SIFT features
Extracting SIFT features
Extracting deep features
Extracting deep features
Fitting rf model
Getting predictions
Accuracy: 0.9603333333333334
Mean accuracy: 0.9603333333333334


In [6]:
from sklearn.decomposition import PCA

# Define different numbers of PCA components to test
n_components_values = [50, 100, 200]

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Run the hybrid model 5 times and compute the mean accuracy for each number of PCA components
mean_accuracies = []
for n_components in n_components_values:
    pca = PCA(n_components=n_components)
    X_train_transformed = pca.fit_transform(X_train_stacked)
    X_test_transformed = pca.transform(X_test_stacked)
    
    accuracies = []
    for _ in range(5):
        # Train the model
        print('Fitting rf model')
        rf_classifier.fit(X_train_transformed, y_train)

        # Evaluate the model on the test set
        print('Getting predictions')
        test_predictions = rf_classifier.predict(X_test_transformed)
        accuracy = accuracy_score(y_test, test_predictions)
        accuracies.append(accuracy)
        print('Accuracy:', accuracy)

    mean_accuracy = np.mean(accuracies)
    mean_accuracies.append(mean_accuracy)
    print(f"Mean accuracy with {n_components} PCA components:", mean_accuracy)

# Print mean accuracies for different numbers of PCA components
print("Mean accuracies:", mean_accuracies)

Mean accuracy with 50 PCA components: 0.9596199999999999
Mean accuracy with 100 PCA components: 0.96214
Mean accuracy with 200 PCA components: 0.96074
Mean accuracies: [0.9596199999999999, 0.96214, 0.96074]
